libs


In [1]:
import pandas as pd
import re
import nltk
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')
from nltk.corpus import stopwords
from Sastrawi.StopWordRemover.StopWordRemoverFactory import StopWordRemoverFactory
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\user\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\user\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\user\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


dataset


In [5]:
# import
DATASET_FILE_NAME = "manual_sentiment_100.csv"
DATASET_FILE_PATH = f"{DATASET_FILE_NAME}"
DATA_FRAME_DATASET = pd.read_csv(DATASET_FILE_PATH)
DATA_FRAME_DATASET_LENGTH = len(DATA_FRAME_DATASET)
print(f"100 sample dataset: ")
display(DATA_FRAME_DATASET.tail(1))

100 sample dataset: 


,user_id,username,user_display_name,tweet_id,full_text,created_at,sentiment_label
99,1.815290e+18,dini_wulan2001,Dini Wulandari,1.840890e+18,@IndonesiaJadi Penajam Paser Utara bakal jadi ...,Mon Sep 30 22:57:54 +0000 2024,positif


text cleaning


In [6]:
def clean_text(text):
  # remove RT tag
  text = re.sub(r'RT\s', '', text)
  # remove @_username
  text = re.sub(r"\@([\w]+)", " ", text)
  # replace emoji decode with space
  text = re.sub(r"\\u[a-zA-Z0-9]{4}", " ", text)
  # replace enter /n/ with space
  text = re.sub(r"\n\s", " ", text)
  text = re.sub(r"\n", " ", text)
  # remove non-ascii
  text = re.sub(r'[^\x00-\x7F]+',' ', text)
  # fix duplicate characters (ex: hellooooo)
  text = re.sub(r'([a-zA-Z])\1\1','\\1', text)
  # replace url
  text = re.sub(r'http[s]?\:\/\/.[a-zA-Z0-9\.\/\_?=%&#\-\+!]+',' ', text)
  text = re.sub(r'pic.twitter.com?.[a-zA-Z0-9\.\/\_?=%&#\-\+!]+',' ', text)
  # convert to lowercase
  text = text.lower()
  # remove hashtag
  text = re.sub(r'\#[a-zA-Z0-9_]+','', text)
  # remove numbers
  text = re.sub(r'[0-9]+',' ', text)
  # remove symbols
  text = re.sub(r'[!$%^&*@#()_+|~=`{}\[\]%\-:";\'<>?,.\/]', ' ', text)
  # remove extra spaces to one space
  text = re.sub(r' +', ' ', text)
  # remove leading and trailing spaces
  text = re.sub(r'^[ ]|[ ]$','', text)
  # replace ikn with ibu kota negara baru
  text = text.replace("ikn", "ibukota negara baru")
  
  return text

DATA_FRAME_DATASET["cleaned_text"] = DATA_FRAME_DATASET["full_text"].apply(clean_text)
display(DATA_FRAME_DATASET["cleaned_text"].tail(1).to_frame())

,cleaned_text
99,penajam paser utara bakal jadi penyuplai panga...


pembakuan kata


In [8]:
SLANG_DICTIONARY_FILE_NAME_1 = "kamus_slang_1.csv"
SLANG_DICTIONARY_FILE_PATH_1 = f"../dictionaries/{SLANG_DICTIONARY_FILE_NAME_1}"
DATA_FRAME_SLANG_DICTIONARY_1 = pd.read_csv(SLANG_DICTIONARY_FILE_PATH_1)

SLANG_DICTIONARY_FILE_NAME_2 = "kamus_slang_2.csv"
SLANG_DICTIONARY_FILE_PATH_2 = f"../dictionaries/{SLANG_DICTIONARY_FILE_NAME_2}"
DATA_FRAME_SLANG_DICTIONARY_2 = pd.read_csv(SLANG_DICTIONARY_FILE_PATH_2)

SLANG_DICTIONARY_1 = pd.Series(DATA_FRAME_SLANG_DICTIONARY_1.formal.values, index=DATA_FRAME_SLANG_DICTIONARY_1.slang).to_dict()
SLANG_DICTIONARY_2 = pd.Series(DATA_FRAME_SLANG_DICTIONARY_2.formal.values, index=DATA_FRAME_SLANG_DICTIONARY_2.slang).to_dict()

def slang_dict_integration_kamus_1(text):
  words = text.split()
  standardization_words = []

  for word in words:
    if word in SLANG_DICTIONARY_1:
      standardization_words.append(SLANG_DICTIONARY_1[word])
    else:
      standardization_words.append(word)

  return " ".join(standardization_words)

def slang_dict_integration_kamus_2(text):
  words = text.split()
  standardization_words = []

  for word in words:
    if word in SLANG_DICTIONARY_2:
      standardization_words.append(SLANG_DICTIONARY_2[word])
    else:
      standardization_words.append(word)

  return " ".join(standardization_words)

DATA_FRAME_DATASET["after_slang_text"] = DATA_FRAME_DATASET["cleaned_text"].apply(slang_dict_integration_kamus_1)
DATA_FRAME_DATASET["after_slang_text"] = DATA_FRAME_DATASET["after_slang_text"].apply(slang_dict_integration_kamus_2)
display(DATA_FRAME_DATASET["after_slang_text"].tail().to_frame())

,after_slang_text
95,jokowi pindah ibu kota bukan keputusan preside...
96,banyak yang belum tahu pemindahan ibu kota ke ...
97,mirip proyek kota baru di lampung ini proyek p...
98,pemindahan ibu kota memberikan peluang besar u...
99,penajam paser utara bakal jadi penyuplai panga...


negation handling


In [9]:
NEGASI_DICTIONARY_FILE_NAME_1 = "negasi.csv"
NEGASI_DICTIONARY_FILE_PATH_1 = f"../dictionaries/{NEGASI_DICTIONARY_FILE_NAME_1}"
DATA_FRAME_NEGASI_DICTIONARY_1 = pd.read_csv(NEGASI_DICTIONARY_FILE_PATH_1)

display(DATA_FRAME_NEGASI_DICTIONARY_1.tail())

ANTONYM_DICTIONARY_FILE_NAME_1 = "antonim_bahasa_indonesia.csv"
ANTONYM_DICTIONARY_FILE_PATH_1 = f"../dictionaries/{ANTONYM_DICTIONARY_FILE_NAME_1}"
DATA_FRAME_ANTONYM_DICTIONARY_1 = pd.read_csv(ANTONYM_DICTIONARY_FILE_PATH_1)

ANTONYM_DICTIONARY_1 = pd.Series(DATA_FRAME_ANTONYM_DICTIONARY_1.antonim.values, index=DATA_FRAME_ANTONYM_DICTIONARY_1.word).to_dict()

display(DATA_FRAME_ANTONYM_DICTIONARY_1.tail())

,negasi
5,tanpa
6,pantang
7,jangan
8,bukanlah
9,sok


,word,antonim
1688,waspada,lalai
1689,ya,"bukan, tidak"
1690,yakin,ragu-ragu
1691,zalim,baik
1692,hanya,banyak


In [10]:
def underscore_negation(text):
  words = text.split()
  negation_words = set(DATA_FRAME_NEGASI_DICTIONARY_1["negasi"].values)
  skip_next = False
  new_words = []
    
  for i in  range(len(words)):
    if skip_next:
      skip_next = False
      continue
    if words[i] in negation_words and i < len(words) - 1:
      new_words.append(words[i] + "_" + words[i+1])
      skip_next = True
    else:
      new_words.append(words[i])

  return " ".join(new_words)

DATA_FRAME_DATASET["underscore_negation_text"] = DATA_FRAME_DATASET["after_slang_text"].apply(underscore_negation)
display(DATA_FRAME_DATASET["underscore_negation_text"].tail().to_frame())

,underscore_negation_text
95,jokowi pindah ibu kota bukan_keputusan preside...
96,banyak yang belum_tahu pemindahan ibu kota ke ...
97,mirip proyek kota baru di lampung ini proyek p...
98,pemindahan ibu kota memberikan peluang besar u...
99,penajam paser utara bakal jadi penyuplai panga...


In [11]:
def swap_antonyms(text):
  words = text.split()
  antonym_dict = dict(zip(DATA_FRAME_ANTONYM_DICTIONARY_1["word"], DATA_FRAME_ANTONYM_DICTIONARY_1["antonim"]))
  new_words = []
    
  for word in words:
    if "_" in word:
      negation, next_word = word.split("_", 1)
      if next_word in antonym_dict:
        new_words.append(antonym_dict[next_word])
      else:
        new_words.append(word)
    else:
      new_words.append(word)
  
  return " ".join(new_words)

DATA_FRAME_DATASET["swap_negation_text"] = DATA_FRAME_DATASET["underscore_negation_text"].apply(swap_antonyms)
display(DATA_FRAME_DATASET["swap_negation_text"].tail().to_frame())

,swap_negation_text
95,jokowi pindah ibu kota bukan_keputusan preside...
96,banyak yang belum_tahu pemindahan ibu kota ke ...
97,mirip proyek kota baru di lampung ini proyek p...
98,pemindahan ibu kota memberikan peluang besar u...
99,penajam paser utara bakal jadi penyuplai panga...


In [12]:
def replace_underscore(text):
  text = re.sub(r'_', ' ', text)

  return text

DATA_FRAME_DATASET["final_negation_text"] = DATA_FRAME_DATASET["swap_negation_text"].apply(replace_underscore)
display(DATA_FRAME_DATASET["final_negation_text"].tail().to_frame())

,final_negation_text
95,jokowi pindah ibu kota bukan keputusan preside...
96,banyak yang belum tahu pemindahan ibu kota ke ...
97,mirip proyek kota baru di lampung ini proyek p...
98,pemindahan ibu kota memberikan peluang besar u...
99,penajam paser utara bakal jadi penyuplai panga...


stopwords removal


In [14]:
SW_DICTIONARY_FILE_NAME_1 = "kamus_stopwords_1.csv"
SW_DICTIONARY_FILE_PATH_1 = f"../dictionaries/{SW_DICTIONARY_FILE_NAME_1}"
DATA_FRAME_SW_DICTIONARY_1 = pd.read_csv(SW_DICTIONARY_FILE_PATH_1)

display(DATA_FRAME_SW_DICTIONARY_1.tail())

custom_stopwords = stopwords.words('indonesian')
custom_stopwords.clear()
custom_stopwords.extend(DATA_FRAME_SW_DICTIONARY_1["stopwords"].values)

factory = StopWordRemoverFactory()
sastrawi_stopwords = factory.get_stop_words()

combined_stopwords = set(custom_stopwords).union(set(sastrawi_stopwords))

def drop_stopwords(text):
  return " ".join([word for word in text.split() if word not in combined_stopwords])

DATA_FRAME_DATASET["after_stopwords_text"] = DATA_FRAME_DATASET["final_negation_text"].apply(drop_stopwords)
display(DATA_FRAME_DATASET["after_stopwords_text"].tail().to_frame())

,stopwords
751,wong
752,yaitu
753,yakin
754,yakni
755,yang


,after_stopwords_text
95,jokowi pindah kota keputusan presiden rakyat k...
96,pemindahan kota ibukota negara jokowi ide soso...
97,proyek kota lampung proyek pemindahan ibukota ...
98,pemindahan kota peluang perkembangan nasional
99,penajam paser utara penyuplai pangan utama ibu...


stemming


In [15]:
factory = StemmerFactory()
stemmer = factory.create_stemmer()

def stem_indonesian_text(text, index):
  return " ".join([stemmer.stem(word) for word in text.split()])

# apply stemming with streaming logs
for index, row in DATA_FRAME_DATASET.iterrows():
  DATA_FRAME_DATASET.at[index, "after_stemming_text"] = stem_indonesian_text(row["after_stopwords_text"], index)

display(DATA_FRAME_DATASET["after_stemming_text"].tail().to_frame())

,after_stemming_text
95,jokowi pindah kota putus presiden rakyat kelua...
96,pindah kota ibukota negara jokowi ide sosok le...
97,proyek kota lampung proyek pindah ibukota prov...
98,pindah kota peluang kembang nasional
99,najam paser utara suplai pangan utama ibukota ...


export


In [16]:
# final column
DATA_FRAME_DATASET["preprocessed_text"] = DATA_FRAME_DATASET["after_stemming_text"]

DATA_FRAME_DATASET.to_csv("100_preprocessing.csv", index=False)